In [118]:
#To split the input line into lines which are start with '+' or '-'.
def split_by_symbol(line):
  
    my_line = '+' + line
    index = 0
    stop_symbol = []
    lines = []
    
    while index < len(my_line):
        if my_line[index] in {'+','-'}:
            stop_symbol.append(index)
        index += 1
    
    for i in range(len(stop_symbol)):
        try:
            lines.append(my_line[stop_symbol[i]:stop_symbol[i+1]])
        except IndexError:
            lines.append(my_line[stop_symbol[i]:len(my_line)])
    
    return lines

In [44]:
#read from position index
def readNumber(line, index):
    number = 0
    while index < len(line) and line[index].isdigit():
        number = number * 10 + int(line[index])
        index += 1
    if index < len(line) and line[index] == '.':
        index += 1
        keta = 0.1
    while index < len(line) and line[index].isdigit():
        number += int(line[index]) * keta
        keta /= 10
        index += 1
    token = {'type': 'NUMBER', 'number': number}
    return token, index

In [27]:
def readPlus(line, index):
  token = {'type': 'PLUS'}
  return token, index + 1

def readMinus(line, index):
  token = {'type': 'MINUS'}
  return token, index + 1

def readMul(line, index):
  token = {'type': 'MUL'}
  return token, index + 1

def readDiv(line, index):
  token = {'type': 'DIV'}
  return token, index + 1


In [28]:
def tokenize(line):
  tokens = []
  index = 0
  while index < len(line):
    if line[index].isdigit():
      (token, index) = readNumber(line, index)
    elif line[index] == '+':
      (token, index) = readPlus(line, index)
    elif line[index] == '-':
      (token, index) = readMinus(line, index)
    elif line[index] == '*':
      (token, index) = readMul(line, index)
    elif line[index] == '/':
      (token, index) = readDiv(line, index)
    else:
      print('Invalid character found: ' + line[index])
      exit(1)
    tokens.append(token)
  return tokens

In [135]:
# calculate the 'x' and '/'
def subevaluate(tokens):

    length = len(tokens)
    index = 2
    if length == 2:
        return tokens[1]['number']
    else:
        answer = tokens[index-1]['number']
        while index<length:
            if tokens[index]['type'] == 'MUL':
                answer *= tokens[index+1]['number']
            if tokens[index]['type'] == 'DIV':
                answer /= tokens[index+1]['number']
            index += 2
    return answer    

In [335]:
def evaluate(line):
  
    lines = split_by_symbol(line)

    answer = 0
    for l in lines:
        token = tokenize(l)
        if token[0]['type'] == 'PLUS':
            answer += subevaluate(token)
        elif token[0]['type'] == 'MINUS':
            answer -= subevaluate(token)
        else:
            print('Invalid syntax')
            exit(1)
    
    return answer

In [340]:
# split the line by parentheses and return the index of the inner () 
def split_by_parentheses(lines):
    
    index = 0
    length = len(lines)
    count_left = 0
    count_right = 0
    left = []
    right = []
    parentheses = []
    
    while index < length:
        if lines[index] == '(':
            left.append(index)
            count_left += 1
        if lines[index] == ')':
            right.append(index)
            count_right += 1
        index += 1
    
    c = 0
    
    if count_left != count_right:
        print('Invalid syntax')
        exit(1)
    
    if count_left == 0:
        return False
    
    while c < count_left:
        if left[c] < right[count_left//2-c]:
            parentheses.append((left[c],right[count_left//2 - c]))
        else:
            parentheses.append((left[c],right[c]))
        c += 1
        
    return parentheses[-1]

In [341]:
def final_evaluate(lines):
    current_lines = lines
    while split_by_parentheses(current_lines):
        location = split_by_parentheses(current_lines)
        splited = current_lines[location[0]+1:location[1]]
        result = evaluate(splited)
        current_lines = current_lines[0:location[0]] + str(result) + current_lines[location[1]+1:]
    
    return evaluate(current_lines)

In [344]:
def test(line):

  actualAnswer = final_evaluate(line)
  expectedAnswer = eval(line)
  if abs(actualAnswer - expectedAnswer) < 1e-8:
    print("PASS! (%s = %f)" % (line, expectedAnswer))
  else:
    print("FAIL! (%s should be %f but was %f)" % (line, expectedAnswer, actualAnswer))


# Add more tests to this function :)
def runTest():
  print("==== Test started! ====")
  test("1+2")
  test("1.0+2.1-3")
  test("(3.0+4*(2-1))/5")
  test("(0+0)/1*0")
  test("(0+0)/(1*0+1)")
  print("==== Test finished! ====\n")

runTest()

while True:
  print('> ', end="")
  line = input()
  answer = final_evaluate(line)
  print("answer = %f\n" % answer)

==== Test started! ====
PASS! (1+2 = 3.000000)
PASS! (1.0+2.1-3 = 0.100000)
PASS! ((3.0+4*(2-1))/5 = 1.400000)
PASS! ((0+0)/1*0 = 0.000000)
PASS! ((0+0)/(1*0+1) = 0.000000)
==== Test finished! ====

> (((3.0+4*(2-1))/5
Invalid syntax


IndexError: list index out of range